In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import enum
import re
import nltk 

from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate  

In [2]:
#Veri setinin yüklenmesi ve örnek veri
dataset = pd.read_csv("data.csv",delimiter=";",header=None,names=["Review","Rating"])
print("Verisetinde {} adet cümle mevcut.".format(len(dataset)))
dataset.head()

Verisetinde 50000 adet cümle mevcut.


,Review,Rating
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [3]:
#Veri ön işleme
#Ön işleme öncesi örnek cümle
print(dataset['Review'].values[0],"\n")

#büyük harflerin küçük harfe çevrilmesi
dataset['Review'] = dataset['Review'].apply(lambda x: x.lower())

data = dataset['Review'].values.tolist()
target = dataset['Rating'].values.tolist()
target = np.array(target)

#Ön işleme sonrası aynı cümle
print(data[0])


One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [4]:
#Cümlelerin içinde geçen kelimelerden 10000 kelimelik bir sözlük oluşturuluyor.
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
#tokenizer.word_index

#Cümleler sayılara dönüştürülüyor
data_tokens = tokenizer.texts_to_sequences(data)

#Cümlelerin önceki ve sonraki hallerinin görüntülenmesi
IDX = 0
print("Öncesi: {}".format(data[IDX]))
print("Sonrası: {}".format(np.array(data_tokens[IDX])))

Öncesi: one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.<br /><br />the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.<br /><br />it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />i would say the main appeal of the show is due to

In [5]:
#RNN'e girdileri vermeden önce tamamının aynı boyutta olması gerekli. Bu sebeple aşağıdaki matematiksel işlemleri yapıyoruz.

num_tokens = np.array([len(tokens) for tokens in data_tokens])
#print(np.mean(num_tokens))
#print(np.std(num_tokens))
#print(np.max(num_tokens))
#print(np.min(num_tokens))

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens) # np.std = standart sapma
max_tokens = int(max_tokens)
print(max_tokens)
#Verinin ne kadarını bu kapsama aldığımızın ölçülmesi
print("%", round(np.sum(num_tokens < max_tokens) / len(num_tokens) * 100, 2))

544
% 94.53


In [6]:
#Padding işlemi. Bulunan uzunluk değerine göre cümlelerin yeniden düzenlenmesi. Kısa olanların başına sıfır eklenmesi.
#Uzun olanlardan baştan silme yapılması
data_pad = pad_sequences(data_tokens, maxlen=max_tokens)

In [7]:
#Modeli oluşuran fonksiyon, KerasClassifier oluşturmak için gerekli
def create_model():
    #RNN oluşturma, ardışık bir model
    model = Sequential()
    
    #her kelimeye karşılık gelen 50 uzunluğunda bir vektör oluşturulur. (Embedding matrisi)
    embedding_size = 50
    
    #matris kelime sayısı ve embedding büyüklüğünde olacak, yani 10bine 50 uzunluğunda 
    model.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size,
                        input_length=max_tokens,
                        name='embedding_layer'))
    #LSTM layerlerinin eklenmesi
    # 16 nöronlu LSTM (16 outputlu , return_sequences=True demek output'un tamamını ver demek)
    model.add(GRU(units=16, return_sequences=True))
    # 8 nöronlu LSTM (8 outputlu , return_sequences=True demek output'un tamamını ver demek)
    model.add(GRU(units=8, return_sequences=True))
    # 4 nöronlu LSTM (4 outputlu , return_sequences=False yani default değer, tek bir output verecek)
    model.add(GRU(units=4))
    # Tek bir nörondan oluşan output layer'ı
    model.add(Dense(1, activation='sigmoid'))

    #modelin derlenmesi 
    #iki sınıf olduğu için loss fonksiyonu olarak binary_crossentropy 
    #modelin başarısını görmek için accuracy metrics
    #optimizasyon algoritması
    optimizer = Adam(lr=1e-3)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy','Precision','Recall',])
    return model

In [8]:
# Modelin değerlendirilmesi
seed=0
#hold-out 
x_train, x_test, y_train, y_test = train_test_split(data_pad, target, test_size=0.1, random_state=seed)
model=create_model()
model.fit(x_train, y_train, epochs=5, batch_size=256, verbose=0)

#Evaluate fonksiyonu yalnızca accuracy ve loss değerini döndürür
result = model.evaluate(x_test, y_test)

import statistics
dizi = [result[2],result[3]]

print("Accuracy= ",result[1])
print("Precision= ",result[2])
print("Recall= ",result[3])
print("F-measure= ",statistics.harmonic_mean(dizi))

157/157 [==============================] - 9s 51ms/step - loss: 0.3191 - accuracy: 0.8794 - precision: 0.8422 - recall: 0.9273
Accuracy=  0.8794000148773193
Precision=  0.8422420024871826
Recall=  0.9272578954696655
F-measure=  0.8827076481939046


In [9]:
# Modelin oluşturulması
# Model eğitimi, bir defa eğitimden geçmesi -> epoch , batch_size -> 256'şar 256'şar beslenecek.
model = KerasClassifier(build_fn=create_model, epochs=5, batch_size=256, verbose=0)

In [10]:
#k-fold cross validation
scoring = ['accuracy', 'precision','recall','f1']
kfold = KFold(n_splits=10, shuffle=False, random_state=seed)
results = cross_validate(model, data_pad, target, cv=kfold, n_jobs=-1, scoring=scoring)

print("Accuracy= ",results['test_accuracy'].mean()," Standart Deviation= ", results['test_accuracy'].std())
print("Precision= ",results['test_precision'].mean()," Standart Deviation= ", results['test_precision'].std())
print("Recall= ",results['test_recall'].mean()," Standart Deviation= ", results['test_recall'].std())
print("F-measure= ",results['test_f1'].mean()," Standart Deviation= ", results['test_f1'].std())

Accuracy=  0.8873  Standart Deviation=  0.00434257987836721
Precision=  0.8885179821252491  Standart Deviation=  0.011887093225371426
Recall=  0.88611221165833  Standart Deviation=  0.01341785311397468
F-measure=  0.8871474438107668  Standart Deviation=  0.003725123099046389


In [11]:
#stratified k-fold validation
scoring = ['accuracy', 'precision','recall','f1']
skfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
results = cross_validate(model, data_pad, target, cv=skfold, n_jobs=-1, scoring=scoring)

print("Accuracy= ",results['test_accuracy'].mean()," Standart Deviation= ", results['test_accuracy'].std())
print("Precision= ",results['test_precision'].mean()," Standart Deviation= ", results['test_precision'].std())
print("Recall= ",results['test_recall'].mean()," Standart Deviation= ", results['test_recall'].std())
print("F-measure= ",results['test_f1'].mean()," Standart Deviation= ", results['test_f1'].std())

Accuracy=  0.88666  Standart Deviation=  0.0076488169019790254
Precision=  0.8921099895644081  Standart Deviation=  0.01971340983941751
Recall=  0.88092  Standart Deviation=  0.028443375327130205
F-measure=  0.885877459109498  Standart Deviation=  0.009093335509276561


In [ ]:
#leave-one-out cross validation
#Don’t Use LOOCV: Large datasets or costly models to fit(e.g. neural networks).
loo = LeaveOneOut()
results = cross_val_score(model, data_pad, target, cv=loo, n_jobs=-1)
print(results.mean())